In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np

# Đọc dữ liệu đã gộp
df = pd.read_excel('table_sample/super_sentinel_merge.xlsx')
# Chọn các đặc trưng và nhãn


features = ['brightness', 'br_ratio', 'min_rgb', 'ndi_rb', 'ndvi']
X = df[features]
y = df['Classvalue']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)



In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(eval_metric='logloss', random_state=42, n_estimators=100)
xgb_model.fit(
    X_train, 
    y_train
)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV

# K-fold giữ tỷ lệ phân phối class (rất quan trọng với dữ liệu Sentinel)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_dis = {
    'n_estimators': np.arange(100, 1000, 150),           # Số cây
    'max_depth': np.arange(8, 13),                       # Độ sâu tối đa
    'learning_rate': loguniform(0.009, 0.18),            # Tốc độ học
    'reg_lambda': loguniform(1e-5, 3),                   # Regularization L2
    'subsample': loguniform(0.5, 1),                     # Tỷ lệ mẫu mỗi cây
    'min_child_weight': loguniform(0.4, 5),              # Tránh overfitting
    'colsample_bytree': loguniform(0.5, 1),              # Số features dùng mỗi cây
    'max_bin': [256, 1024],                              # Số bin cho histogram
    'tree_method': ['hist'],                             # Dùng hist để tăng tốc (CPU)
    # 'device': ['cuda'],                                # Bỏ nếu không có GPU
    'nthread': [2],                                      # Số luồng CPU
    'eval_metric': ['logloss']                           # Hàm đánh giá phân loại
}

# Khởi tạo mô hình tìm kiếm
search = RandomizedSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, random_state=42),
    param_distributions=param_dis,
    n_iter=100,
    scoring='f1',
    refit=True,
    cv=cv,
    verbose=3,
    n_jobs=-1
)

In [ ]:
search.fit(X_train, y_train)

print("Best params:", search.best_params_)
print("Best F1 score:", search.best_score_)

In [ ]:
best_xgb = XGBClassifier(
    colsample_bytree= 0.9236117361774273,
    eval_metric='logloss',
    learning_rate=0.010330890615126487,
    max_bin=256,
    max_depth=9,
    min_child_weight=0.4372916896505061,
    n_estimators=250,
    nthread=2,
    reg_lambda=0.00029434882880863505,
    subsample=0.5379579346349724,
    tree_method='hist',
    random_state=42
)

# Train the model
best_xgb.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)


print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
import joblib
joblib.dump(best_rf, 'best_xgb_model.pkl')